# Vector Fitting Examples

The Vector Fitting feature is demonstrated using to three example networks from the scikit-rf `data` folder. Additional explanations and background information can be found in the Vector Fitting tutorial.

## Example 1: Ring Slot

To create a `VectorFitting` instance, a `Network` containing the frequency responses of the N-port is passed. In this first example the *ring slot* is used:

In [ ]:
import skrf
from skrf.vectorFitting import VectorFitting
import numpy as np

nw = skrf.data.ring_slot
vf = VectorFitting(nw)

Now, the vector fit can be performed. The number of poles has to be specified, which depends on the *behaviour* of the responses. A smooth response would only require very few poles (2-5). In this case, 2 poles are sufficient:

In [ ]:
vf.vectorfit(n_poles_real=2, n_poles_cmplx=0)

As shown in the output, the pole relocation process converged quickly after just 4 iteration steps. The fitted model parameters are now stored in the class variables `poles`, `zeros`, `proportional_coeff` and `constant_coeff` for further use. To verify the result, the model response can be compared to the original network response. As the model will return a response at any given frequency, it makes sense to also check its response outside the frequency range of the original samples:

In [ ]:
freqs = np.linspace(0, 200e9, 201)

vf.plot_fit_linear(0, 0, freqs) # compare S11
vf.plot_fit_linear(1, 0, freqs) # compare S21

To use the model in a circuit simulation, an equivalent circuit can be created based on the fitting parameters. This is currently only implemented for SPICE, but the structure of the equivalent circuit can be adopted to any kind of circuit simulator.

In [ ]:
vf.write_spice_subckt_s('./equiv_subskt.sp')